# PyCity Schools Analysis

- Your analysis here

---

In [21]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [22]:
# Calculate the total number of unique schools
school_count = school_data_complete["school_name"].nunique()
student_count = school_data_complete["Student ID"].nunique()
total_budget = school_data["budget"].sum()
average_math_score = school_data_complete["math_score"].mean()
average_reading_score = school_data_complete["reading_score"].mean()


In [23]:
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["Student ID"]
passing_math_percentage = passing_math_count / float(student_count) * 100


In [24]:
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()[("Student ID")]
passing_reading_percentage = passing_reading_count / float(student_count) * 100


In [25]:
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["Student ID"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100


In [26]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "Overall Passing Rate": [overall_passing_rate]
})
# Formatting
district_summary["Total Schools"] = district_summary["Total Schools"].map("{:,}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.6f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.6f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:,.6f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:,.6f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{:,.6f}%".format)

# Display the DataFrame
print(district_summary)


  Total Schools Total Students    Total Budget Average Math Score  \
0            15         39,170  $24,649,428.00          78.985371   

  Average Reading Score % Passing Math % Passing Reading Overall Passing Rate  
0             81.877840     74.980853%        85.805463%           65.172326%  


## School Summary

In [27]:
# Use the code provided to select the type per school; the total student count per school; and the total school budget from school_data
per_school_summary = school_data.set_index("school_name")[["type", "size", "budget"]]
per_school_summary.head(2)


,type,size,budget
school_name,,,
Huang High School,District,2917,1910635
Figueroa High School,District,2949,1884411


In [28]:
# Calculate the total student count per school from school_data
per_school_summary.rename(columns={"type": "School Type", "size": "Total Students", "budget": "Total School Budget"}, inplace=True)


In [29]:
# Calculate the total school budget and per capita spending per school from school_data
per_school_summary["Per Student Budget"] = per_school_summary["Total School Budget"] / per_school_summary["Total Students"]

In [30]:
per_school_math_score = school_data_complete.groupby("school_name")["math_score"].mean()
per_school_summary["Average Math Score"] = per_school_math_score
per_school_reading_score = school_data_complete.groupby("school_name")["reading_score"].mean()
per_school_summary["Average Reading Score"] = per_school_reading_score


In [31]:
# Calculate the number of students per school with math scores of 70 or higher from school_data_complete
school_students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby("school_name")["Student ID"].count()
per_school_summary["% Passing Math"] = school_students_passing_math /per_school_summary["Total Students"] * 100

In [32]:
# Calculate the number of students per school with reading scores of 70 or higher from school_data_complete
school_students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("school_name")["Student ID"].count()
per_school_summary["% Passing Reading"] = school_students_passing_reading /per_school_summary["Total Students"] * 100

In [33]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()


In [34]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_both = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_both = students_passing_both.groupby(["school_name"]).size()
per_school_summary["% Overall Passing"] = school_students_passing_both /per_school_summary["Total Students"] * 100

In [35]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary["Average Math Score"] = per_school_summary["Average Math Score"].map("{:,.3f}%".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:,.3f}%".format)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].map("{:,.3f}%".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:,.3f}%".format)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:,.3f}%".format)

# Display the DataFrame
per_school_summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629%,81.183%,65.684%,81.316%,53.514%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712%,81.158%,65.988%,80.739%,53.204%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359%,83.726%,93.867%,95.855%,89.892%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290%,80.934%,66.753%,80.863%,53.528%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351%,83.817%,93.392%,97.139%,90.599%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274%,83.989%,93.868%,96.540%,90.583%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062%,83.976%,94.133%,97.040%,91.335%
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048%,81.034%,66.680%,81.933%,54.642%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803%,83.815%,92.506%,96.253%,89.227%


## Highest-Performing Schools (by % Overall Passing)

In [36]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062%,83.976%,94.133%,97.040%,91.335%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418%,83.849%,93.272%,97.309%,90.948%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351%,83.817%,93.392%,97.139%,90.599%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274%,83.989%,93.868%,96.540%,90.583%
Pena High School,Charter,962,"$585,858.00",$609.00,83.840%,84.045%,94.595%,95.946%,90.541%


## Bottom Performing Schools (By % Overall Passing)

In [37]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("% Overall Passing", ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.843%,80.745%,66.367%,80.220%,52.988%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712%,81.158%,65.988%,80.739%,53.204%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629%,81.183%,65.684%,81.316%,53.514%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290%,80.934%,66.753%,80.863%,53.528%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072%,80.966%,66.058%,81.222%,53.539%


## Math Scores by Grade

In [38]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby("school_name")["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby("school_name")["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby("school_name")["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby("school_name")["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_math_scores,
    "10th": tenth_grader_math_scores,
    "11th": eleventh_grader_math_scores,
    "12th": twelfth_grader_math_scores
})

math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [39]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby("school_name")["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby("school_name")["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby("school_name")["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby("school_name")["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
    "9th": ninth_grade_reading_scores,
    "10th": tenth_grader_reading_scores,
    "11th": eleventh_grader_reading_scores,
    "12th": twelfth_grader_reading_scores
})

reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [40]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]


In [41]:
# Create a copy of the school summary for later aggregations
school_spending_df = per_school_summary.copy()


In [42]:
# Use `pd.cut` on the per_school_capita Series from earlier to categorize per student spending based on the bins.
per_school_capita = school_spending_df["Per Student Budget"].replace('[$,]', '', regex=True).astype(float)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, bins=spending_bins, labels=labels)

school_spending_df.index.name = None
school_spending_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629%,81.183%,65.684%,81.316%,53.514%,$645-680
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712%,81.158%,65.988%,80.739%,53.204%,$630-645
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359%,83.726%,93.867%,95.855%,89.892%,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290%,80.934%,66.753%,80.863%,53.528%,$645-680
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351%,83.817%,93.392%,97.139%,90.599%,$585-630
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274%,83.989%,93.868%,96.540%,90.583%,<$585
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062%,83.976%,94.133%,97.040%,91.335%,<$585
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048%,81.034%,66.680%,81.933%,54.642%,$585-630
Holden High School,Charter,427,"$248,087.00",$581.00,83.803%,83.815%,92.506%,96.253%,89.227%,<$585
Pena High School,Charter,962,"$585,858.00",$609.00,83.840%,84.045%,94.595%,95.946%,90.541%,$585-630


In [43]:
def clean_percentage(column):
    # Ensure the column is of string type
    column = column.astype(str)
    # Strip '%' and convert to float
    return column.str.replace('%', '', regex=False).astype(float) / 100

In [44]:
#  Remove string symbols from the data so that it is calcuable.
school_spending_df["Average Math Score"] = clean_percentage(school_spending_df["Average Math Score"])
school_spending_df["Average Reading Score"] = clean_percentage(school_spending_df["Average Reading Score"])
school_spending_df["% Passing Math"] = clean_percentage(school_spending_df["% Passing Math"])
school_spending_df["% Passing Reading"] = clean_percentage(school_spending_df["% Passing Reading"])
school_spending_df["% Overall Passing"] = clean_percentage(school_spending_df["% Overall Passing"])

In [45]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Math Score"].mean() * 100
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["Average Reading Score"].mean() * 100
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Math"].mean() * 100
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Passing Reading"].mean() * 100
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"], observed=True)["% Overall Passing"].mean() * 100
school_spending_df.head(2)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Huang High School,District,2917,"$1,910,635.00",$655.00,0.76629,0.81183,0.65684,0.81316,0.53514,$645-680
Figueroa High School,District,2949,"$1,884,411.00",$639.00,0.76712,0.81158,0.65988,0.80739,0.53204,$630-645


In [46]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})
# Formatting
spending_summary["Average Math Score"] = spending_summary["Average Math Score"].map("{:,.3f}%".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:,.3f}%".format)
spending_summary["% Passing Math"] = spending_summary["% Passing Math"].map("{:,.3f}%".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:,.3f}%".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:,.3f}%".format)
# Display results
spending_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455%,83.934%,93.460%,96.611%,90.369%
$585-630,81.900%,83.156%,87.133%,92.718%,81.418%
$630-645,78.519%,81.624%,73.484%,84.392%,62.858%
$645-680,76.997%,81.028%,66.165%,81.134%,53.527%


## Scores by School Size

In [47]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [48]:
# Create a copy of the school summary for later aggregations
school_size_df = per_school_summary.copy()

In [49]:
# Use `pd.cut` on the per_school_counts Series from earlier to categorize school size based on the bins.
per_school_counts = school_size_df["Total Students"]
school_size_df["School Size"] = pd.cut(per_school_counts, bins=size_bins, labels=size_labels)

school_size_df.index.name = None

# Convert School Size to a string
school_size_df["School Size"] = school_size_df["School Size"].astype(str)
school_size_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629%,81.183%,65.684%,81.316%,53.514%,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712%,81.158%,65.988%,80.739%,53.204%,Large (2000-5000)
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359%,83.726%,93.867%,95.855%,89.892%,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290%,80.934%,66.753%,80.863%,53.528%,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351%,83.817%,93.392%,97.139%,90.599%,Medium (1000-2000)
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274%,83.989%,93.868%,96.540%,90.583%,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062%,83.976%,94.133%,97.040%,91.335%,Medium (1000-2000)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048%,81.034%,66.680%,81.933%,54.642%,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803%,83.815%,92.506%,96.253%,89.227%,Small (<1000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.840%,84.045%,94.595%,95.946%,90.541%,Small (<1000)


In [50]:
# Remove string symbols from the data so that it is calcuable.
school_size_df["Average Math Score"] = clean_percentage(school_size_df["Average Math Score"])
school_size_df["Average Reading Score"] = clean_percentage(school_size_df["Average Reading Score"])
school_size_df["% Passing Math"] = clean_percentage(school_size_df["% Passing Math"])
school_size_df["% Passing Reading"] = clean_percentage(school_size_df["% Passing Reading"])
school_size_df["% Overall Passing"] = clean_percentage(school_size_df["% Overall Passing"])

In [51]:
# Calculate averages for the desired columns.
size_math_scores = school_size_df.groupby(["School Size"])["Average Math Score"].mean() *100
size_reading_scores = school_size_df.groupby(["School Size"])["Average Reading Score"].mean() * 100
size_passing_math = school_size_df.groupby(["School Size"])["% Passing Math"].mean() * 100
size_passing_reading = school_size_df.groupby(["School Size"])["% Passing Reading"].mean() * 100
size_overall_passing = school_size_df.groupby(["School Size"])["% Overall Passing"].mean() * 100


In [52]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
})

# Formatting
size_summary["Average Math Score"] = size_summary["Average Math Score"].map("{:,.3f}%".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:,.3f}%".format)
size_summary["% Passing Math"] = size_summary["% Passing Math"].map("{:,.3f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:,.3f}%".format)
size_summary["% Overall Passing"] = size_summary["% Overall Passing"].map("{:,.3f}%".format)
# Display results
size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Large (2000-5000),77.746%,81.344%,69.963%,82.766%,58.286%
Medium (1000-2000),83.374%,83.865%,93.599%,96.791%,90.621%
Small (<1000),83.821%,83.930%,93.550%,96.100%,89.884%


## Scores by School Type

In [53]:
school_type_df = per_school_summary.copy()

In [54]:
# Remove string symbols from the data so that it is calcuable.
school_type_df["Average Math Score"] = clean_percentage(school_type_df["Average Math Score"])
school_type_df["Average Reading Score"] = clean_percentage(school_type_df["Average Reading Score"])
school_type_df["% Passing Math"] = clean_percentage(school_type_df["% Passing Math"])
school_type_df["% Passing Reading"] = clean_percentage(school_type_df["% Passing Reading"])
school_type_df["% Overall Passing"] = clean_percentage(school_type_df["% Overall Passing"])

In [55]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = school_type_df.groupby(["School Type"])["Average Math Score"].mean() * 100
average_reading_score_by_type = school_type_df.groupby(["School Type"])["Average Reading Score"].mean() * 100
average_percent_passing_math_by_type = school_type_df.groupby(["School Type"])["% Passing Math"].mean() * 100
average_percent_passing_reading_by_type = school_type_df.groupby(["School Type"])["% Passing Reading"].mean() * 100
average_percent_overall_passing_by_type = school_type_df.groupby(["School Type"])["% Overall Passing"].mean() *100


In [56]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing": average_percent_overall_passing_by_type
})

# Formatting
type_summary["Average Math Score"] = type_summary["Average Math Score"].map("{:,.3f}%".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:,.3f}%".format)
type_summary["% Passing Math"] = type_summary["% Passing Math"].map("{:,.3f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:,.3f}%".format)
type_summary["% Overall Passing"] = type_summary["% Overall Passing"].map("{:,.3f}%".format)


# Display results
type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.474%,83.897%,93.621%,96.587%,90.432%
District,76.957%,80.967%,66.549%,80.799%,53.672%
